In [1]:
import pandas as pd
import numpy as np

In [113]:
def generate_series():
    times = pd.date_range('2020-01-01', '2020-2-01', freq='MS')
    sample_size = 2
    groups_full = ['A', 'B']
    
    series_length = len(times) * sample_size * len(groups_full)
    index_names_full = ['time', 'samples', 'group']

    iterables = [times, range(sample_size), groups_full]
    df_multi_index = pd.MultiIndex.from_product(iterables, names=index_names_full)
    
    A = pd.Series(data=np.tile([1, 2], 4), index=df_multi_index, name='A')
    B = pd.Series(data=np.tile([1, 2], 4)+2, index=df_multi_index, name='B')
    C = pd.Series(data=np.full(int(series_length/2), 5),
                  index=pd.MultiIndex.from_product([times, range(sample_size)], names=['time', 'samples']),
                  name='C')
    return A, B, C


In [114]:
A, B, C = generate_series()
A,B,C

(time        samples  group
 2020-01-01  0        A        1
                      B        2
             1        A        1
                      B        2
 2020-02-01  0        A        1
                      B        2
             1        A        1
                      B        2
 Name: A, dtype: int64,
 time        samples  group
 2020-01-01  0        A        3
                      B        4
             1        A        3
                      B        4
 2020-02-01  0        A        3
                      B        4
             1        A        3
                      B        4
 Name: B, dtype: int64,
 time        samples
 2020-01-01  0          5
             1          5
 2020-02-01  0          5
             1          5
 Name: C, dtype: int64)

In [115]:
# first, experimenting with simple operations without changing index levels
def df_ops(X, Y):
    res = pd.DataFrame()
    res['X'] = X
    res['Y'] = Y
    res['X+Y'] = X + Y
    res['X-Y'] = X - Y
    res['X*Y'] = X * Y
    res['X/Y'] = X / Y
    res['Y+X'] = Y + X
    res['Y-X'] = Y - X
    res['Y*X'] = Y * X
    res['Y/X'] = Y / X
    return res

def df_ops2(X, Y):
    res = pd.DataFrame()
    res['X'] = X
    res['Y'] = Y
    res['X+Y'] = X + Y
    res['Y+X'] = Y + X
    res['X-Y'] = X - Y
    res['Y-X'] = Y - X
    res['X*Y'] = X * Y
    res['Y*X'] = Y * X
    res['X/Y'] = X / Y
    res['Y/X'] = Y / X
    res['~1'] = res['X/Y'] * res['Y/X']
    return res


# this looks correct
df_ops2(A, B)

X  Y  X+Y  Y+X  X-Y  Y-X  X*Y  Y*X       X/Y  Y/X  \
time       samples group                                                      
2020-01-01 0       A      1  3    4    4   -2    2    3    3  0.333333  3.0   
                   B      2  4    6    6   -2    2    8    8  0.500000  2.0   
           1       A      1  3    4    4   -2    2    3    3  0.333333  3.0   
                   B      2  4    6    6   -2    2    8    8  0.500000  2.0   
2020-02-01 0       A      1  3    4    4   -2    2    3    3  0.333333  3.0   
                   B      2  4    6    6   -2    2    8    8  0.500000  2.0   
           1       A      1  3    4    4   -2    2    3    3  0.333333  3.0   
                   B      2  4    6    6   -2    2    8    8  0.500000  2.0   

                           ~1  
time       samples group       
2020-01-01 0       A      1.0  
                   B      1.0  
           1       A      1.0  
                   B      1.0  
2020-02-01 0       A      1.0  
                   B      1.0  
           1       A      1.0  
                   B      1.0

In [141]:
# next, experiment with changed index level order

A = A # full series ordered ['time', 'samples', 'group']
B = B # full series ordered ['time', 'samples', 'group']
C = C # countries only ordered ['time', 'samples']
D = A.reorder_levels(['group', 'samples', 'time'])
E = A.reorder_levels(['group', 'time', 'samples'])
F = A.reorder_levels(['samples', 'group', 'time'])
G = A.reorder_levels(['samples', 'time', 'group'])
H = A.reorder_levels(['time', 'group', 'samples'])
I = C.reorder_levels(['samples', 'time'])

df_ops2(A, C) # expected results
df_ops2(B, C) # expected results
df_ops2(D, C) # all NaN except X
df_ops2(E, C) # all NaN except X
df_ops2(F, C) # all NaN except X
df_ops2(G, C) # all Y NaN; ops beginning with Y are NaN
df_ops2(H, C) # all NaN except X

df_ops2(A, I) # all Y NaN; ops beginning with Y are NaN
df_ops2(B, I) # all Y NaN; ops beginning with Y are NaN
df_ops2(C, I) # all Y NaN; ops beginning with Y are NaN
df_ops2(D, I) # all NaN except X
df_ops2(E, I) # all NaN except X
df_ops2(F, I) # all NaN except X
df_ops2(G, I) # expected results
df_ops2(H, I) # all NaN except X

df_ops2(A, D) # all Y NaN; ops beginning with Y are NaN
df_ops2(A, E) # all Y NaN; ops beginning with Y are NaN
df_ops2(A, F) # all Y NaN; ops beginning with Y are NaN
df_ops2(A, G) # all Y NaN; ops beginning with Y are NaN
df_ops2(A, H) # all Y NaN; ops beginning with Y are NaN

df_ops2(F, G)

X   Y  X+Y  Y+X  X-Y  Y-X  X*Y  Y*X  X/Y  Y/X  ~1
samples group time                                                         
0       A     2020-01-01  1 NaN    2  NaN    0  NaN    1  NaN  1.0  NaN NaN
        B     2020-01-01  2 NaN    4  NaN    0  NaN    4  NaN  1.0  NaN NaN
1       A     2020-01-01  1 NaN    2  NaN    0  NaN    1  NaN  1.0  NaN NaN
        B     2020-01-01  2 NaN    4  NaN    0  NaN    4  NaN  1.0  NaN NaN
0       A     2020-02-01  1 NaN    2  NaN    0  NaN    1  NaN  1.0  NaN NaN
        B     2020-02-01  2 NaN    4  NaN    0  NaN    4  NaN  1.0  NaN NaN
1       A     2020-02-01  1 NaN    2  NaN    0  NaN    1  NaN  1.0  NaN NaN
        B     2020-02-01  2 NaN    4  NaN    0  NaN    4  NaN  1.0  NaN NaN